<a href="https://colab.research.google.com/github/diazid/sales-predictions/blob/main/sales_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1 - Part 2: Food Sales Prediction

Name: Israel Diaz



**Description:**

This project will be a sales prediction for food items sold at various stores. The goal of this is to help the retailer understand the properties of products and outlets that play crucial roles in increasing sales.

## Loading Data

Loading data from container

In [ ]:
filepath = 'https://drive.google.com/uc?export=download&id=1apwZQiYRcktux62Ki6qaJa_JI-hDGb75'

In [ ]:
#IMPORTING PANDAS LIBRARY
import pandas as pd

In [ ]:
#LOADING DATA INTO PANDAS DATAFRAME
df = pd.read_csv(filepath)

Previewing the content and info.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


I see that there is a couple of the variables that has missing values:

* No. Entries : 8523
* `Item Weight` : 7060 non null values
* `Outlet_Size` : 6113 non null values

Other variables are complete. 

In [ ]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [ ]:
df.shape

(8523, 12)

The data frame register 8523 entries and 12 variable. Each variable represent the following. 



|Variable Name            	| Description |
|---------------------------|-------------|
|Item_Identifier            |	Unique product ID|
|Item_Weight	              | Weight of product|
|Item_Fat_Content           |	Whether the product is low fat or regular|
|Item_Visibility          	|The percentage of total display area of all products in a store allocated to the particular product|
|Item_Type	                |The category to which the product belongs|
|Item_MRP	                  |Maximum Retail Price (list price) of the product|
|Outlet_Identifier          |Unique store ID|
|Outlet_Establishment_Year	|The year in which store was established|
|Outlet_Size	              |The size of the store in terms of ground area covered|
|Outlet_Location_Type	      |The type of area in which the store is located|
|Outlet_Type	              |Whether the outlet is a grocery store or some sort of supermarket|
|Item_Outlet_Sales         	|Sales of the product in the particular store. This is the target variable to be predicted. |

In the next section I will be dealing with the data preparation.

## Data Cleaning

### Data types

In [ ]:
df.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

All data types seems to be correct, at least in the first watch. If in the next steps I find I have to change in the data types it will be done. 

### Finding Duplicates

In [ ]:
df.duplicated().sum()

0

There is no duplicated entries in the data set.

### Solving Missing Values

In [ ]:
df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

There are have missing values in `Item_Weight` and `Outlet_type` columns. We'll be exploring that columns in the following steps

I decided to frop the column `Item_Weight` due to the following reasons:

* They are weight values. 
* They are mostly unique values for each item. 

In [ ]:
# Dropping the Item Weight column
df.drop(columns=['Item_Weight'], inplace= True)

Other columns that presented missing values is `Outlet_Size` variable:

In [ ]:
df['Outlet_Size'].value_counts()

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64

There are three categories in the `Outlet_Size` variable:
* Small
* Medium
* High

I'll try to undestand why are the reasons of the missing values:

1. The first instance is that `Small` and `Medium` categories exceed in more than 2000 entries to each other, but the `High` categorie no even reach 1000. 
2. Despite the previous reason will be necessary to perform an in-depth exploration to understand what is happening within the data. 

#### Looking for relationships in the data.

I'll explore the relations that exist between `Outlet_Size` and other variables, as follow:

* `Item_Type`
* `Outlet_type`
* `Outlet_Identifier`
* `Item_Fat_Content`
* `Outlet_Location_Type`

I have choosen this variables because they are all categorical variables. 


#### `Outlet_Size` vs `Item_Type`

In [ ]:
df.groupby(by=['Item_Type'])['Outlet_Size'].value_counts(normalize=True)

Item_Type              Outlet_Size
Baking Goods           Medium         0.438445
                       Small          0.403888
                       High           0.157667
Breads                 Medium         0.463687
                       Small          0.396648
                       High           0.139665
Breakfast              Medium         0.455696
                       Small          0.379747
                       High           0.164557
Canned                 Medium         0.460722
                       Small          0.401274
                       High           0.138004
Dairy                  Medium         0.439516
                       Small          0.399194
                       High           0.161290
Frozen Foods           Medium         0.445528
                       Small          0.404878
                       High           0.149593
Fruits and Vegetables  Medium         0.467724
                       Small          0.371461
                       Hi

* For every Item category there is less gat between Small and Medium categories, on the contrary we see a wide gap with the High category.  
* The normalized values shown that the High category is from 13% to 17% of the entire data, while Small and High are the between the 87% to the 83#, or at least 42% each.

#### `Outlet_Size` vs `Outlet_Type`

In [ ]:
df.groupby(by=['Outlet_Type'])['Outlet_Size'].value_counts(normalize= True)

Outlet_Type        Outlet_Size
Grocery Store      Small          1.000000
Supermarket Type1  Small          0.499731
                   High           0.250403
                   Medium         0.249866
Supermarket Type2  Medium         1.000000
Supermarket Type3  Medium         1.000000
Name: Outlet_Size, dtype: float64

* I can't conclude any relationship between the variables that are confronting. Because I don't see the same insight in the last pair of variables. 

#### `Outlet_Size` vs `Outlet_Identifier`

In [ ]:
df.groupby(by=['Outlet_Identifier'])['Outlet_Size'].value_counts(normalize= True)

Outlet_Identifier  Outlet_Size
OUT013             High           1.0
OUT018             Medium         1.0
OUT019             Small          1.0
OUT027             Medium         1.0
OUT035             Small          1.0
OUT046             Small          1.0
OUT049             Medium         1.0
Name: Outlet_Size, dtype: float64

* Also, there is imposible to determine any relationship with this variable pair. 

#### `Outlet_Size` vs `Item_Fat_Content`

In [ ]:
df.groupby(by=['Item_Fat_Content'])['Outlet_Size'].value_counts(normalize= True)

Item_Fat_Content  Outlet_Size
LF                Medium         0.450644
                  Small          0.412017
                  High           0.137339
Low Fat           Medium         0.455919
                  Small          0.391376
                  High           0.152705
Regular           Medium         0.458574
                  Small          0.390655
                  High           0.150771
low fat           Medium         0.419753
                  Small          0.358025
                  High           0.222222
reg               Medium         0.512195
                  Small          0.329268
                  High           0.158537
Name: Outlet_Size, dtype: float64

Here I see a view of the same insigh of the first two pairs of variables analyzed, but before giving some conclution is necesary to treat the inconsistencies in the categories. I will solve this as follow

#### Solving preliminary inconsistencies within the categories in the `Item_Fat_Content` variable

In [ ]:
df['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular', 'low fat', 'LF', 'reg'], dtype=object)

I will assume that the category `LF` is the same as `Low Fat`, as well as `reg` to `Regular`. And the will be added capt to `low fat`. 

In [ ]:
df['Item_Fat_Content'].replace({'LF': 'Low Fat', 
                                'low fat': 'Low Fat', 
                                'reg': 'Regular'}, 
                                inplace=True)

df['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular'], dtype=object)

Once this is corrected, I can continue with the previus analysis between `Outlet_Size` and `Item_Fat_Content`

#### `Outlet_Siz`e vs `Item_Fat_Content` (Cont.)

In [ ]:
df.groupby(by=['Item_Fat_Content'])['Outlet_Size'].value_counts(normalize= True)

Item_Fat_Content  Outlet_Size
Low Fat           Medium         0.454867
                  Small          0.391909
                  High           0.153224
Regular           Medium         0.460612
                  Small          0.388323
                  High           0.151066
Name: Outlet_Size, dtype: float64

Now I can see more clearly that is happening the same thing than the first pair of variables. 


#### `Outlet_Size` vs `Outlet_Location_Type`

In [ ]:
df.groupby(by=['Outlet_Location_Type'])['Outlet_Size'].value_counts(normalize= True)

Outlet_Location_Type  Outlet_Size
Tier 1                Small          0.610553
                      Medium         0.389447
Tier 2                Small          1.000000
Tier 3                Medium         0.666547
                      High           0.333453
Name: Outlet_Size, dtype: float64

With this pair is difficult to see any relationshitp that helps me determine how to assign the missing values of this variable. 

Breafly resuming the conclutions:

* There is no relationship that allow me to make a decison about where to impute the missing values between `Outlet_Size` and the [`Outlet_Type`, `Outlet_Identifier`, `Outlet_Location_Type`] variables.
* On the contrary, I can see relationship or correlation on the categories between `Outlet_Size` and [`Item_Type`,`Item_Fat_Content`].

There are to options of imputing the missing values here:

* Create a new category, it could call 'undefined' for example.
* Impute the missing values to the 'High' category and analize how is the distribution.

I'll continue with the 2nd one, and if I see any problems whith the analysis I always be able to return to implement a new solution. 

#### Imputing missing values to `High` category.

First of all I'll create a copy of the dataframe in case I have to return and take the other option of create a new category.


In [ ]:
# IMPUTING MISSING VALUES TO HIGH CATEGORY

#Copy the original dataframe
df_2 = df.copy(deep=True)

#imputing the missing values to High
df_2['Outlet_Size'].fillna('High', inplace=True)

#### Checking the data with imputed values

In [ ]:
df_2.isna().sum()

Item_Identifier              0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

In [ ]:
df_2['Outlet_Size'].unique()

array(['Medium', 'High', 'Small'], dtype=object)

In [ ]:
df_2.groupby(by=['Item_Type'])['Outlet_Size'].value_counts(normalize=True)

Item_Type              Outlet_Size
Baking Goods           High           0.398148
                       Medium         0.313272
                       Small          0.288580
Breads                 High           0.386454
                       Medium         0.330677
                       Small          0.282869
Breakfast              High           0.400000
                       Medium         0.327273
                       Small          0.272727
Canned                 High           0.374422
                       Medium         0.334361
                       Small          0.291217
Dairy                  High           0.390029
                       Medium         0.319648
                       Small          0.290323
Frozen Foods           High           0.389019
                       Medium         0.320093
                       Small          0.290888
Fruits and Vegetables  High           0.398539
                       Medium         0.335227
                       Sm

In [ ]:
df_2.groupby(by=['Item_Fat_Content'])['Outlet_Size'].value_counts(normalize= True)

Item_Fat_Content  Outlet_Size
Low Fat           High           0.392967
                  Medium         0.326083
                  Small          0.280950
Regular           High           0.390552
                  Medium         0.330672
                  Small          0.278776
Name: Outlet_Size, dtype: float64

* I am going forward with this data as I see that the one seems more uniform.
* I always could return and take the other path in case of the model present high amount of error.  

### Looking for Inconsistencies

I previous started this with the `Item_Fat_Content` variable, but I want to see all the variables in case more of them be found.

In [ ]:
df_2['Item_Identifier'].unique()

array(['FDA15', 'DRC01', 'FDN15', ..., 'NCF55', 'NCW30', 'NCW05'],
      dtype=object)

In [ ]:
df_2['Outlet_Establishment_Year'].unique()

array([1999, 2009, 1998, 1987, 1985, 2002, 2007, 1997, 2004])

In [ ]:
df_2['Outlet_Identifier'].unique()

array(['OUT049', 'OUT018', 'OUT010', 'OUT013', 'OUT027', 'OUT045',
       'OUT017', 'OUT046', 'OUT035', 'OUT019'], dtype=object)

In [ ]:
df_2['Outlet_Location_Type'].unique()

array(['Tier 1', 'Tier 3', 'Tier 2'], dtype=object)

In [ ]:
df_2['Outlet_Type'].unique()

array(['Supermarket Type1', 'Supermarket Type2', 'Grocery Store',
       'Supermarket Type3'], dtype=object)

I see no more inconsistencies

### Summary statistics

In [ ]:
df.describe()

,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,8523.000000,8523.000000,8523.000000,8523.000000
mean,0.066132,140.992782,1997.831867,2181.288914
std,0.051598,62.275067,8.371760,1706.499616
min,0.000000,31.290000,1985.000000,33.290000
25%,0.026989,93.826500,1987.000000,834.247400
50%,0.053931,143.012800,1999.000000,1794.331000
75%,0.094585,185.643700,2004.000000,3101.296400
max,0.328391,266.888400,2009.000000,13086.964800


## Exploratory Visuals

## Explanatory Visuals